
# Statevec

Here we take benchmarking for remove_qubit method in Statevec class.

This benchmark measures the time taken to remove single qubit from a statevector.
The methods we use are the followings:
    1. :meth:`~graphix.sim.statevec.Statevec.remove_qubit`
        Optimized method to remove single qubit.
    2. :meth:`~graphix.sim.statevec.Statevec.ptrace`
        Old method to remove multiple qubits.
    3. :meth:`~graphix.pattern.Pattern.simulate_pattern`
        Circuit simulation using pattern matching.


Firstly, let us import relevant modules:



In [ ]:
from time import perf_counter
import matplotlib.pyplot as plt
import numpy as np
from graphix import Circuit
from graphix.clifford import CLIFFORD_MUL
from graphix.sim.statevec import Statevec, StatevectorBackend, meas_op

Next, we define the simulation runner:



In [ ]:
def run(backend):
    """Perform MBQC simulation"""
    for cmd in backend.pattern.seq:
        if cmd[0] == "N":
            backend.add_nodes([cmd[1]])
        elif cmd[0] == "E":
            backend.entangle_nodes(cmd[1])
        elif cmd[0] == "M":
            backend.measure(cmd)
        elif cmd[0] == "X":
            backend.correct_byproduct(cmd)
        elif cmd[0] == "Z":
            backend.correct_byproduct(cmd)
        elif cmd[0] == "C":
            backend.apply_clifford(cmd)
        else:
            raise ValueError("invalid commands")
        if backend.pattern.seq[-1] == cmd:
            backend.finalize()

Then, we define a random circuit generator:



In [ ]:
def random_circuit(nqubit, depth, seed=None):
    r"""Generate a random circuit.

    This function generates a random circuit with nqubit qubits and depth layers.
    The circuit is generated by CNOT gates and RZ gates.

    Parameters
    ----------
    nqubit : int
        number of qubits
    depth : int
        number of layers
    seed : int
        random seed

    Returns
    -------
    circuit : graphix.transpiler.Circuit object
        generated circuit
    """
    if seed is not None:
        np.random.seed(seed)
    qubit_index = [i for i in range(nqubit)]
    circuit = Circuit(nqubit)
    for _ in range(depth):
        np.random.shuffle(qubit_index)
        for j in range(len(qubit_index) // 2):
            circuit.cnot(qubit_index[2 * j], qubit_index[2 * j + 1])
        for j in range(len(qubit_index)):
            circuit.rz(qubit_index[j], 2 * np.pi * np.random.random())
    return circuit

We define the test cases: shallow (depth=1) random circuits, only changing the number of qubits.



In [ ]:
test_cases = [(i, 1) for i in range(2, 24)]

pattern_time = []
circuit_time = []

We then run simulations.
First, we run the pattern simulations with :meth:`~graphix.sim.statevec.Statevec.remove_qubit`
and circuit simulation:



In [ ]:
for case in test_cases:
    nqubit, depth = case
    circuit = random_circuit(nqubit, depth)
    pattern = circuit.transpile()
    pattern.standardize()
    pattern.minimize_space()
    max_qubit_num = 20 if nqubit < 20 else 50
    backend = StatevectorBackend(pattern, max_qubit_num=max_qubit_num)
    print(f"max space for nqubit={nqubit} circuit is ", pattern.max_space())
    start = perf_counter()
    run(backend)
    end = perf_counter()
    print(f"nqubit: {nqubit}, depth: {depth}, time: {end - start}")
    pattern_time.append(end - start)
    start = perf_counter()
    circuit.simulate_statevector()
    end = perf_counter()
    circuit_time.append(end - start)

To test old method :meth:`~graphix.sim.statevec.Statevec.ptrace`,
we need to overwrite the class :class:`~graphix.sim.statevec.StatevectorBackend`'



In [ ]:
class OldStatevectorBackend(StatevectorBackend):
    def add_nodes(self, nodes):
        """add new qubit to internal statevector
        and assign the corresponding node number
        to list self.node_index.
        Parameters
        ----------
        nodes : list of node indices
        """
        if not self.state:
            self.state = Statevec(nqubit=0)
        n = len(nodes)
        sv_to_add = Statevec(nqubit=n)
        self.state.tensor(sv_to_add)
        self.node_index.extend(nodes)
        self.Nqubit += n
        if self.Nqubit == self.max_qubit_num:
            self.trace_out()

    def measure(self, cmd):
        """Perform measurement of a node in the internal statevector and trace out the qubit

        Parameters
        ----------
        cmd : list
            measurement command : ['M', node, plane angle, s_domain, t_domain]
        """
        # choose the measurement result randomly
        result = np.random.choice([0, 1])
        self.results[cmd[1]] = result

        # extract signals for adaptive angle
        s_signal = np.sum([self.results[j] for j in cmd[4]])
        t_signal = np.sum([self.results[j] for j in cmd[5]])
        angle = cmd[3] * np.pi
        if len(cmd) == 7:
            vop = cmd[6]
        else:
            vop = 0
        if int(s_signal % 2) == 1:
            vop = CLIFFORD_MUL[1, vop]
        if int(t_signal % 2) == 1:
            vop = CLIFFORD_MUL[3, vop]
        m_op = meas_op(angle, vop=vop, plane=cmd[2], choice=result)
        loc = self.node_index.index(cmd[1])
        self.state.evolve_single(m_op, loc)

        self.to_trace.append(cmd[1])
        self.to_trace_loc.append(loc)

    def finalize(self):
        """to be run at the end of pattern simulation."""
        self.trace_out()
        self.sort_qubits()
        self.state.normalize()

    def trace_out(self):
        """trace out the qubits buffered in self.to_trace from self.state"""
        self.state.normalize()
        self.state.ptrace(self.to_trace_loc)
        for node in self.to_trace:
            self.node_index.remove(node)
        self.Nqubit -= len(self.to_trace)
        self.to_trace = []
        self.to_trace_loc = []

Lastly, run the simulation with the old method :meth:`~graphix.sim.statevec.Statevec.ptrace`:



In [ ]:
test_cases_old = [(i, 1) for i in range(2, 11)]

old_pattern_time = []

for case in test_cases_old:
    nqubit, depth = case
    circuit = random_circuit(nqubit, depth)
    pattern = circuit.transpile()
    pattern.standardize()
    pattern.minimize_space()
    print(f"max space for nqubit={nqubit} circuit is ", pattern.max_space())
    old_backend = OldStatevectorBackend(pattern)
    start = perf_counter()
    run(old_backend)
    end = perf_counter()
    print(f"nqubit: {nqubit}, depth: {depth}, time: {end - start}")
    old_pattern_time.append(end - start)

plot the pattern_time



In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.scatter([case[0] for case in test_cases_old], old_pattern_time, label="pattern simulator with ptrace(old)")
ax.scatter([case[0] for case in test_cases], pattern_time, label="pattern simulator with remove_qubit(new)")
ax.scatter([case[0] for case in test_cases], circuit_time, label="circuit simulator")
ax.set(
    xlabel="nqubit",
    ylabel="time (s)",
    yscale="log",
    title="Time to simulate random circuits",
)
fig.legend(bbox_to_anchor=(0.6, 0.9))
fig.show()